In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   # 提供 boxplot，基于 Plots

include("Data_Generation_PP.jl")
include("Estimation_PP.jl")
include("Evaluation_PP.jl")
# # include("Plot_Figures.jl")
include("Implement_All_Methods_PP.jl")
include("Models_PP.jl")

Solve_RO (generic function with 1 method)

In [2]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/")
if !isdir(data_dir)
    mkpath(data_dir)
end

## Functions

In [3]:
function set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)
    Params = Dict()
    Params["N"] = N
    Params["N_u"] = N_u
    Params["K"] = K
    Params["S_train"] = S_train
    Params["S_test"] = S_test
    Params["offdiag_sign"] = offdiag_sign
    Params["max_offdiag"] = max_offdiag
    Params["P_bar"] = P_bar
    Params["RO_coef_all"] = RO_coef_all
    Params["seed"] = seed
    return Params
end

set_Params (generic function with 1 method)

## Evaluate

#### Baseline Parameters

In [14]:
seed = 1;
N = 5;
N_u = 1;
K = 10;
S_train = 200;
S_test = 1000;
P_bar = 5.0;
iterations = 10;
# RO_coef_all = [0.0];
RO_coef_all = [0.0,0.01,0.05,0.1,0.12,0.15,0.2,0.25,0.3];
Random.seed!(seed)
project_dir = "Joint_Pricing_and_Promotion/"

"Joint_Pricing_and_Promotion/"

### offdiag is zero

In [15]:
offdiag_sign = "mix";
max_offdiag = 1.0;
Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed);

In [16]:
sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
this_data_file = string(data_dir,project_dir,sub_file_name)
if !isdir(this_data_file)
    mkpath(this_data_file)
end
save(string(this_data_file, "Params.jld2"), Params);
println(this_data_file)

/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Joint_Pricing_and_Promotion/N=5_N_u=1_K=10_S_train=200_offdiag_sign=mix_max_offdiag=1.0/


- Generate Data

In [17]:
Input_Data = generate_Input_Data(S_train,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);

****** iter = 1 ********
****** iter = 2 ********
****** iter = 3 ********
****** iter = 4 ********
****** iter = 5 ********
****** iter = 6 ********
****** iter = 7 ********
****** iter = 8 ********
****** iter = 9 ********
****** iter = 10 ********


- Run Oracle

In [18]:
print_flag=true
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

iter=1: rev_Oracle = 2.330864, price_Oracle = [3.62, 4.58, 4.73, 4.82, 2.4]
iter=2: rev_Oracle = 1.879285, price_Oracle = [1.95, 4.52, 3.47, 2.52, 5.0]
iter=3: rev_Oracle = 1.999128, price_Oracle = [4.09, 4.29, 4.84, 0.55, 2.44]
iter=4: rev_Oracle = 1.952183, price_Oracle = [4.17, 2.01, 4.87, 4.53, 4.66]
iter=5: rev_Oracle = 1.917262, price_Oracle = [0.32, 4.95, 4.1, 2.08, 4.3]
iter=6: rev_Oracle = 1.14296, price_Oracle = [1.47, 4.89, 0.6, 4.87, 1.43]
iter=7: rev_Oracle = 2.789089, price_Oracle = [4.6, 4.41, 0.47, 4.74, 3.6]
iter=8: rev_Oracle = 1.334824, price_Oracle = [4.23, 4.34, 0.02, 0.43, 1.42]
iter=9: rev_Oracle = 1.757298, price_Oracle = [1.2, 1.05, 4.73, 2.46, 2.09]
iter=10: rev_Oracle = 3.37768, price_Oracle = [4.94, 4.19, 3.83, 4.97, 4.84]


- Run ETO

In [19]:
print_flag = true
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

iter=1: rev_ETO = 2.282881, price_ETO = [3.62, 3.4, 4.56, 4.82, 2.4]
iter=2: rev_ETO = 1.803063, price_ETO = [3.16, 4.52, 3.47, 2.52, 5.0]
iter=3: rev_ETO = 1.394116, price_ETO = [0.91, 4.29, 2.53, 4.35, 1.84]
iter=4: rev_ETO = 1.420697, price_ETO = [4.17, 3.55, 4.87, 2.95, 4.66]
iter=5: rev_ETO = 0.130835, price_ETO = [3.3, 0.33, 4.07, 4.68, 1.21]
iter=6: rev_ETO = 0.005847, price_ETO = [3.16, 3.13, 1.83, 0.11, 3.84]
iter=7: rev_ETO = 0.137918, price_ETO = [4.45, 4.41, 2.1, 1.45, 4.39]
iter=8: rev_ETO = 0.809802, price_ETO = [4.23, 0.03, 4.64, 2.99, 0.86]
iter=9: rev_ETO = 0.882353, price_ETO = [4.74, 3.92, 4.73, 2.46, 1.1]
iter=10: rev_ETO = 3.289654, price_ETO = [4.94, 4.19, 4.01, 4.97, 4.84]


- Run RO under different uncertainty set

In [20]:
gamma_list = [0.0]
# RO_coef_all = [0.0,0.01,0.05,0.1,0.12,0.15,0.2,0.25,0.3];
dual_norm = 2
psi_lb_coef = -10.0
phi_lb_coef = -10.0

-10.0

In [ ]:
RST_RO = Run_RO(gamma_list, dual_norm,iterations, N, N_u, K, Input_Data,print_flag,psi_lb_coef,phi_lb_coef);

In [13]:
gamma = 0.0
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO = [RST_RO["iter=$(iter)_RST"]["Rev_gamma=$(gamma)"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
println("Rev_RO/Rev_Oracle = ",round.(Rev_RO./Rev_Oracle,digits=4))

Rev_ETO/Rev_Oracle = [0.9977, 0.9869, 0.9421, 0.9917, 1.0, 0.849, 0.9769, 0.9985, 0.7981, 1.0]
Rev_RO/Rev_Oracle = [0.9977, 0.9869, 0.9421, 0.9917, 1.0, 0.849, 0.9769, 0.9985, 0.7981, 1.0]


### offdiag is positive

In [ ]:
# offdiag_sign = "positive";
# Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [ ]:
# sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
# this_data_file = string(data_dir,project_name,sub_file_name)
# if !isdir(this_data_file)
#     mkpath(this_data_file)
# end
# save(string(this_data_file, "Params.jld2"), Params);

In [ ]:
# Random.seed!(seed)
# Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
# Input_Data = Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data);
# save(string(this_data_file, "Input_Data.jld2"), Input_Data);

In [ ]:
# print_flag=true
# RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
# save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [ ]:
# RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
# println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
# save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [ ]:
# model_name="Two_Side"
# RST_RO = Dict(); Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     println("Running RO with RO_coef = ",RO_coef)
#     RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
#     RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
# end
# save(string(this_data_file, "RST_RO.jld2"), RST_RO);

In [ ]:
# for RO_coef in RO_coef_all
#     Rev_RO_this = Rev_RO["RO_coef=$(RO_coef)"]
#     println("Average = ",round(mean(Rev_RO_this./Rev_ETO),digits=4),", Rev_RO/Rev_ETO for RO_coef=$(RO_coef) = ",round.(Rev_RO_this./Rev_ETO,digits=4),)
# end

### offdiag is negative

In [ ]:
# offdiag_sign = "negative";
# Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [ ]:
# sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
# this_data_file = string(data_dir,project_name,sub_file_name)
# if !isdir(this_data_file)
#     mkpath(this_data_file)
# end
# save(string(this_data_file, "Params.jld2"), Params);

In [ ]:
# print_flag=true
# RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
# save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [ ]:
# RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
# println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
# save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [ ]:
# model_name="Two_Side"
# RST_RO = Dict(); Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     println("Running RO with RO_coef = ",RO_coef)
#     RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
#     RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
# end
# save(string(this_data_file, "RST_RO.jld2"), RST_RO);